In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd

### 駿台

In [2]:
MOSHI3 = {"第３回駿台全国模試":"http://www2.sundai.ac.jp/yobi/sv/sundai/moshi_P/place_PD/1337374601732.html", 
         "大学入試センター試験プレテスト":"http://www2.sundai.ac.jp/yobi/sv/sundai/moshi_P/place_PD/1337364967005.html"}
DATE3 = {"第３回駿台全国模試":"2017-12-03", 
         "大学入試センター試験プレテスト":"2017-12-17"}

In [4]:
GRADE3 = []
for m in MOSHI3:
    print(m, '...')
    f = urlopen(MOSHI3[m])
    soup = BeautifulSoup(f, 'html.parser')
    for p in soup.find("table", class_="standard-table").findAll("tr")[1:]:
        place = p.a.get_text()
        # print(p.a.get_text(), p.a.attrs['href'])
        driver = webdriver.PhantomJS()
        driver.get('http://www2.sundai.ac.jp' + p.a.attrs['href'])
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()
        try:
            geo = soup.iframe.attrs['src'].split('!2d')[1].split('!3m')[0].split('!3d')
        except:
            geo = [None, None]
        
        GRADE3.append([m, 3, DATE3[m], place, geo[1], geo[0]])

第３回駿台全国模試 ...
大学入試センター試験プレテスト ...


In [5]:
MOSHI2 = {"第1回高2駿台全国マーク模試":"http://www2.sundai.ac.jp/yobi/sv/sundai/moshi_P/place_PD/1337364967166.html"}
DATE2 = {"第1回高2駿台全国マーク模試":"2017-12-23"}
GRADE2 = []
for m in MOSHI2:
    f = urlopen(MOSHI2[m])
    soup = BeautifulSoup(f, 'html.parser')
    for p in soup.find("table", class_="standard-table").findAll("tr")[1:]:
        place = p.a.get_text()
        # print(p.a.get_text(), p.a.attrs['href'])
        driver = webdriver.PhantomJS()
        driver.get('http://www2.sundai.ac.jp' + p.a.attrs['href'])
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()
        try:
            geo = soup.iframe.attrs['src'].split('!2d')[1].split('!3m')[0].split('!3d')
        except:
            geo = [None, None]
        
        GRADE2.append([m, 2, DATE2[m], place, geo[1], geo[0]])

In [7]:
dat = pd.concat([pd.DataFrame(GRADE3, columns=['moshi', 'grade', 'date', 'place', 'latitude', 'longitude']), 
                 pd.DataFrame(GRADE2, columns=['moshi', 'grade', 'date', 'place', 'latitude', 'longitude'])])
dat.head()

,moshi,grade,date,place,latitude,longitude
0,第３回駿台全国模試,3,2017-12-03,駿台予備学校 札幌校,43.067795,141.346643
1,第３回駿台全国模試,3,2017-12-03,駿台予備学校仙台校,38.2582664,140.8840019
2,第３回駿台全国模試,3,2017-12-03,茨城県職業人材育成センター,None,None
3,第３回駿台全国模試,3,2017-12-03,宇都宮ビジネス電子専門学校,None,None
4,第３回駿台全国模試,3,2017-12-03,小野池學院,None,None


In [8]:
dat.to_csv('res/sundai.csv')